In [82]:
# Dataset : https://www.kaggle.com/datasets/gauravduttakiit/ants-bees?resource=download
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [83]:
transforms={'train':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
    'val':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [84]:
data_dir='data/archive/hymenoptera_data'
#image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),transform=transforms[x]) for x in ['train','val']}


In [85]:
image_datasets={}
image_datasets['train']=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=transforms['train'])
image_datasets['val']=datasets.ImageFolder(os.path.join(data_dir,'val'),transform=transforms['val'])

In [86]:
data_loader={}
data_loader['train']=DataLoader(image_datasets['train'],batch_size=32,shuffle=True)
data_loader['val']=DataLoader(image_datasets['val'],batch_size=32,shuffle=True)

In [87]:
model=models.resnet18(pretrained=True)
#print(model)
model_ftrs=model.fc.in_features
model.fc = nn.Sequential()
model.fc.ft1=nn.Linear(model_ftrs,out_features=256)
model.fc.ft2=nn.Linear(256,out_features=64)
model.fc.ft3=nn.Linear(64,out_features=16)
model.fc.ft4=nn.Linear(16,out_features=2)

criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.0001)


In [88]:
for epoch in range(30):
    for input,label in data_loader['train']:
        output=model(input)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss.item())


0.633276104927063
0.5703212022781372
0.5007633566856384
0.42448192834854126
0.4712880551815033
0.4319295883178711
0.2997700572013855
0.25353726744651794
0.47874945402145386
0.3552878499031067


In [89]:
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for input,label in data_loader['val']:
        output=model(input)
        _,predicted=torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicted==label).sum().item()
    print('Accuracy: {}%'.format(100*correct/total))
    print(total, correct)

Accuracy: 86.9281045751634%
153 133
